# Post-MVP Modeling

The purpose of this notebook is to improve the final model aka "Minimum Viable Product" that predicted with an F1 of 0.3955.

In [3]:
# importing relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
%matplotlib inline
plt.style.use('bmh')
import pickle

# NLP libraries
import nltk
from sklearn.feature_extraction import text 
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# modeling libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from collections import Counter
from sklearn import metrics, utils, model_selection, svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, plot_confusion_matrix, classification_report

In [4]:
# custom fuctions

import sys
sys.path.insert(0, '../src')
# module_path = sys.path.insert(0, '../src')

# if module_path not in sys.path:
#     sys.path.append(module_path)

import helper_functions as helpers

## Method 1: Linear SVM with Count Vectorization

### Importing X and y from `nlp_preprocesing.ipynb`

In [5]:
X_lem = pickle.load(open('../pickle/X_lem.pkl', 'rb'))
y_lem = pd.read_pickle('../pickle/y_lem.pkl')

In [6]:
# setting up stop words
stop_words = set(stopwords.words('english'))

### TTS & CountVectorize

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_lem, y_lem, test_size=0.20, random_state=20)

In [7]:
# using count vectorizor
count = CountVectorizer(stop_words=stopwords)

## Method 2: Offensive Language as Stop Words